In [34]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords
import csv
import time
import io
import hashlib
import requests

stopwords = set(stopwords.words('spanish'))
political_keys = r"C:\Users\meiza\Documents\GitHub\Forum-Politization\Dataset\political_keys.csv"
dataset = r"C:\Users\meiza\Documents\GitHub\Forum-Politization\Dataset\dataset.csv"

In [35]:
#A PARTIR DE UN HILO DEVOLVEMOS TÍTULO Y MENSAJES

def Webscrap(hilo):
    #time.sleep(1)
  
    #REALIZAMOS LA LLAMADA
    root_url = "https://www.forocoches.com/foro/showthread.php?t="
        
    page_html = s.get(root_url+hilo)
    page_soup = soup(page_html.content, 'html5lib')


        
    #COGEMOS LOS MENSAJES Y TITULO
    #page_soup = soup(page_html, "html5lib")
    titulo = page_soup.findAll("span", {"class": "cmega"})

    if not(titulo == []):
        titulo = titulo[0].text


        mensajes = page_soup.findAll("div", {"id": lambda x: x and x.startswith('post_message')})

    
        #QUITAMOS QUOTES
        i = 0
        for mensaje in mensajes:
            if (mensaje.div != None):
                mensaje.div.extract()
            #SÓLO EL TEXTO
            mensajes[i] = mensaje.text
        
            #QUITAMOS LOS PARRAFOS Y DOBLES ESPACIOS
            mensajes[i] = mensajes[i].replace('\n', ' ').replace('\r', '').lower()
            mensajes[i] = ' '.join(mensajes[i].split())
            i += 1


        return titulo,mensajes
    else:
        return None,None

In [36]:
#NORMALIZAMOS EL TEXTO
def Normalize(text):
    #QUITAMOS ACENTOS
        
    
    #QUITAMOS \n y \r. TEXTO EN MINÚSCULA
    text = text.replace('\n', ' ').replace('\r', '').lower()
    
    #QUITAMOS NÚMEROS
    text = ''.join([i for i in text if not i.isdigit()])
    
    #QUITAMOS PUNTUACIÓN
    text = "".join(l for l in text if l not in (string.punctuation,'?','¿',',','.'))
    
    #QUITAMOS DOBLES ESPACIOS
    text = ' '.join(text.split())

    #STEMMING Y QUITAMOS STOPWORDS
    stemmer = SnowballStemmer("spanish")
    words = text.split()
    text = ''
    for word in words:
        if not word in stopwords:
            stemmed_word = stemmer.stem(word)
            text = text + " " + stemmed_word
    
    return text

In [37]:
#FUNCIÓN PARA VER SI EL TÍTULO INDICA QUE ES UN HILO POLÍTICO

def Ispolitical(titulo):
    political = False
    
    #Normalizamos titulo
    titulo = Normalize(titulo)
    titulo = titulo.split()

    #Cruzamos nuestro título con nuestras palabras clave para saber si es título político
    with open(political_keys, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            key = Normalize(row[0]).strip()
            for palabra in titulo:
                if (key == palabra):
                    political = True
    return political    

In [38]:
#FUNCIÓN PARA ECSRIBIR EN EL DATASET

def Writedatasetpolitico(mensajes):
    with io.open(dataset, 'a', newline='',encoding="utf-8") as ds:
        dataset_writer = csv.writer(ds, delimiter=',', quotechar="'", quoting=csv.QUOTE_NONNUMERIC)
        for mensaje in mensajes:
            dataset_writer.writerow([1, mensaje])

def Writedatasetnopolitico(mensajes):
    with io.open(dataset, 'a', newline='',encoding="utf-8") as ds:
        dataset_writer = csv.writer(ds, delimiter=',', quotechar="'", quoting=csv.QUOTE_NONNUMERIC)
        for mensaje in mensajes:
            dataset_writer.writerow([0, mensaje])

In [40]:
#INICIAMOS SESIÓN
s = requests.Session()
username = '<USERNAME>'
password = '<PASSWORD>'
BASE_URL = 'https://www.forocoches.com/foro'
    
r = s.post(BASE_URL + '/login.php?do=login', {
'vb_login_username':        username,
'vb_login_password':        password,
'vb_login_md5password':     hashlib.md5(password.encode()).hexdigest(),
'vb_login_md5password_utf': hashlib.md5(password.encode("utf-8")).hexdigest(),

'cookieuser': 1,
'do': 'login',
's': '',
'securitytoken': 'guest'
})

In [122]:
#BUSCADOR DE HILOS POLITIZADOS PARA AÑADIRLOS AL DATASET

hilo_inicio=7996396
hilo_final=7996500

for i in range(hilo_inicio, hilo_final+1):
    titulo, mensajes = Webscrap(str(i))
    if titulo != None:
        politico = Ispolitical(titulo)
        if politico:
            print(titulo)
            Writedatasetpolitico(mensajes)

A ver si os enteráis que no vais a coger la paguita tan fácil palurdos
Risas manifa negros Madrid
Alguna vez te han comido el anillo negro?


In [72]:
#AÑADIR HILOS AL DATASET MANUALMENTE
hilo='7781086&page=11'
titulo, mensajes = Webscrap(hilo)
Writedatasetpolitico(mensajes)
print(titulo)

Clara Ponsati y Puigdemont en Twitter + nazis
